In [1]:
import pandas as pd
from datetime import date
import import_ipynb
import pipeline as pipeline

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Data successfully loaded into DataFrame.
Filtered data successfully saved to 'enrollment_master.csv'.


/home/sarvesh/miniconda3/envs/timetabling/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Data successfully loaded into DataFrame.
Filtered data successfully saved to 'exam_master.csv'.
exam_choice.csv written successfully.
course: DHS 101B
['Lab Exam']
exam_choice.csv written successfully.
{'CSE 102': {'Muralidhara V N': ['CSE 102-A'], 'Pradeesha Ashok': ['CSE 102-B']}, 'CSE 102 P': {'Muralidhara V N': ['CSE 102 P-A'], 'Vivek Yadav': ['CSE 102 P-B']}, 'CSE 104': {'Manisha Kulkarni': ['CSE 104-A'], 'Prem Singh': ['CSE 104-B']}, 'DAS 101': {'Uttam Kumar': ['DAS 101-B'], 'Raghuram Bharadwaj': ['DAS 101-A']}, 'DAS 101P': {'Vinu E V': ['DAS 101P-B'], 'Chandrashekar Ramanathan': ['DAS 101P-A']}, 'DHS 101B': {'Amit Prakash': ['DHS 101B-B', 'DHS 101B-A']}, 'DHS 201': {'Sumanth S': ['DHS 201-A'], 'TBA': ['DHS 201-A', 'DHS 201-B']}, 'EGC 121': {'Karthikeyan Vaidyanathan': ['EGC 121-A'], 'Ananda Y R': ['EGC 121-B']}, 'EGC 123': {'Ajay Bakre': ['EGC 123-A'], 'Amrita Mishra': ['EGC 123-B']}}
Files updated successfully.
Files updated successfully.
{'AIM 101': 120.0, 'AIM 102': 180.0, 'A

## Time Slots

In [2]:
# Time Slots
days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri','Sat']
n_days = len(days)
times = ['08:00', '08:15', '08:30', '08:45',
         '09:00', '09:15', '09:30', '09:45',
         '10:00', '10:15', '10:30', '10:45',
         '11:00', '11:15', '11:30', '11:45',
         '12:00', '12:15', '12:30', '12:45', '13:00',
         '14:00', '14:15', '14:30', '14:45',
         '15:00', '15:15', '15:30', '15:45',
         '16:00', '16:15', '16:30', '16:45',
         '17:00', '17:15', '17:30', '17:45', '18:00']
n_times = len(times)
lunch_time_index = times.index('13:00')
pre_lunch_times = list(range(lunch_time_index + 1))
post_lunch_times = list(range((lunch_time_index + 1), len(times)))

def begin_end_pairs(session_len:int) -> list:
    pairs = []
    for s in range(0, len(times) - session_len):
        e = s + session_len
        if (((s in pre_lunch_times) and (e in pre_lunch_times)) or
            ((s in post_lunch_times) and (e in post_lunch_times))):
            pairs.append((s,e))
    return pairs
# No lectures start/end in these time slots (as (day, time) tuples) --- labs, tutorials are OK
mon_2pm = (days.index('Mon'), times.index('14:00'),)
wed_2pm = (days.index('Wed'), times.index('14:00'),)
no_lecture_slots = ([(mon_2pm[0], t,) for t in range(mon_2pm[1], (mon_2pm[1] + 6))] +
                    [(wed_2pm[0], t,) for t in range(wed_2pm[1], (wed_2pm[1] + 6))])

def overlap(b:int, e:int, t:int) -> bool:
    # Returns a bool indicating whether the intervel (b,e) contains time t
    # b,e,t are indexes into the 'times' array
    assert (0 <= b < n_times) and(0 <= e < n_times) and (0 <= t < n_times)
    return (b <= t <= e)



## Classrooms

In [3]:
# classroom_cols = {'RoomNum': str, 'Capacity': int, 'Category': str,
#                   'HasProjector': bool, 'HasWacom': bool}
classroom_cols = {'RoomNumber': str, 'Capacity': int}
classrooms = pd.read_csv('classroom_master.csv', dtype=classroom_cols)
n_rooms = classrooms.shape[0]

#get room capacity given room id 
def get_room_capacity(room_id: int) -> int:
    return int(classrooms.loc[room_id, 'Capacity'])

def get_room_category(room_id: int) -> str:
    a=classrooms.loc[room_id, 'Lab']
    b=classrooms.loc[room_id, 'DeptLab']
    if a:
        if b:
            return 'CSE Lab'
        return 'ECE Lab'
    return 'NotLab'

def get_room_num(room_id: int) -> str:
    return classrooms.loc[room_id, 'RoomNumber']

def check_cse_lab_room(room_id: int) -> int:
    if get_room_category(room_id) == 'CSE Lab':
        return True
    return False

def check_ece_lab_room(room_id: int) -> int:
    if get_room_category(room_id) == 'ECE Lab':
        return True
    return False

def get_total_cse_lab_capacity() -> int:
    total_capacity = 0
    for i in range(n_rooms):
        if check_cse_lab_room(i):
            total_capacity += get_room_capacity(i)
    return total_capacity

def get_total_ece_lab_capacity() -> int:
    total_capacity = 0
    for i in range(n_rooms):
        if check_ece_lab_room(i):
            total_capacity += get_room_capacity(i)
    return total_capacity

def get_total_not_lab_capacity() -> int:
    total_capacity = 0
    for i in range(n_rooms):
        if not check_cse_lab_room(i) and not check_ece_lab_room(i):
            total_capacity += get_room_capacity(i)

def get_course_dept(course : str) -> str:
    course = course[0:3]
    if course =="ECE" or course =="VLS" or course =="COM":
        return "ECE"
    return "CSE"

## Courses & Enrollment

In [16]:
courses = pd.read_csv('course_master.csv')
n_courses = courses.shape[0]
enrollment = pd.read_csv('enrollment_master.csv')
n_enrollments = enrollment.shape[0]
n_students = enrollment['EmailID'].nunique()
students = enrollment['EmailID'].unique()

exams = pd.read_csv('exam_master.csv')

course_id = {}
for i in range(n_courses):
    course_id[courses['Code'][i]] = i

course_id_code={}
for i in range(n_courses):
    course_id_code[i] = courses['Code'][i]

def get_course_id(course_code: str) -> int:
    return course_id[course_code.strip()]

def get_course_strength(course_id: int) -> int:
    return int(enrollment[enrollment['CourseCode'] == courses['Code'][course_id]].shape[0])

def get_courses_student(student_id: int) -> list:
    l =  enrollment[enrollment['EmailID'] == students[student_id]]['CourseCode'].values
    return [get_course_id(x) for x in l]
    
def get_course_title(course_id: int) -> str:
    return courses.loc[course_id, 'Title']

def get_faculty1(course_id: int) -> str:
    return exams.loc[course_id, 'Faculty1']
def get_faculty2(course_id: int) -> str:
    return exams.loc[course_id, 'Faculty2']

isCore = {}

for iter,row in enrollment.iterrows():
    isCore[get_course_id(row['CourseCode'])] = row['OTHER/REGULAR-ELECTIVE/REGULAR-CORE'] == 'REGULAR-CORE'

def get_isCore(course_id: int) -> bool:
    return isCore[course_id]





def get_course_category(course_id: int) -> str:
    
    a = courses.loc[course_id,'Lab']
    b = courses.loc[course_id,'Dept']
    if a:
        if b == 'CSE':
            return 'CSE Lab'
        return 'ECE Lab'
    return 'NotLab'

def get_dept(course_id: int) -> str:
    return courses.loc[course_id, 'Dept']

def get_not_rooms_course(course_id: int) -> list:
    l=[]
    for i in range(n_rooms):
        if get_room_category(i) != get_course_category(course_id):
            l.append(i)
    return l

course_group = {}   # course --> cat
for c in range(n_courses):
    course_group[c] = get_course_category(c)

print("course_group",course_group)

group_by_cat = {} # cat --> [rooms]
for r in range(n_rooms):
    cat = get_room_category(r)
    if cat not in group_by_cat:
        group_by_cat[cat] = []
    group_by_cat[cat].append(r)

print("group_by_cat",group_by_cat) 

#  room --> [courses]
courses_in_room = {}
for r in range(n_rooms):
    courses_in_room[r] = []

for c in range(n_courses):
    for r in range(n_rooms):
        if get_room_category(r) == get_course_category(c):
            courses_in_room[r].append(c)

def get_course_duration(course_id: int) -> int:
    return courses.loc[course_id, 'Exam Duration']

def get_preferred_days(course_id: int) -> list:
    # return courses.loc[course_id, 'Preferred Days'].split(',')
    code=course_id_code[course_id]
    l = pipeline.course_code_prefdays[code]
    res=[days.index(x) for x in l]
    return res

print(get_preferred_days(1))




def get_preferred_times(course_id: int) -> list:
    code=course_id_code[course_id]
    l = pipeline.course_code_preftime[code]
    res=[times.index(x[:-3]) for x in l]
    return res

print(get_preferred_times(1))

def get_preferred_slots(course_id: int) -> list:
    x = courses.loc[course_id, 'Preferred Timings'].split(',')
    triplets = []
    for i in range(0, len(x), 3):
        begin = x[i].strip(" [(\"'")
        end = x[i+1].strip(" )'\"]")
        day = x[i+2].strip(" )'\"]")
        triplets.append((begin, end, day))
    for i in range(len(triplets)):
        triplets[i] = (times.index(triplets[i][0]), times.index(triplets[i][1]), days.index(triplets[i][2]))
    return triplets

print([get_room_category(x) for x in group_by_cat[course_group[0]]])


course_group {0: 'NotLab', 1: 'NotLab', 2: 'NotLab', 3: 'NotLab', 4: 'CSE Lab', 5: 'NotLab', 6: 'NotLab', 7: 'NotLab', 8: 'NotLab', 9: 'NotLab', 10: 'NotLab', 11: 'NotLab', 12: 'NotLab', 13: 'NotLab', 14: 'NotLab', 15: 'NotLab', 16: 'NotLab', 17: 'NotLab', 18: 'NotLab', 19: 'NotLab', 20: 'NotLab', 21: 'NotLab', 22: 'NotLab', 23: 'NotLab', 24: 'NotLab', 25: 'NotLab', 26: 'NotLab', 27: 'NotLab', 28: 'NotLab', 29: 'NotLab', 30: 'NotLab', 31: 'NotLab', 32: 'NotLab', 33: 'ECE Lab', 34: 'NotLab', 35: 'ECE Lab', 36: 'NotLab', 37: 'NotLab', 38: 'NotLab', 39: 'NotLab', 40: 'NotLab', 41: 'NotLab', 42: 'NotLab', 43: 'NotLab', 44: 'NotLab', 45: 'NotLab', 46: 'NotLab', 47: 'NotLab', 48: 'CSE Lab', 49: 'CSE Lab', 50: 'CSE Lab', 51: 'CSE Lab', 52: 'NotLab', 53: 'NotLab', 54: 'NotLab', 55: 'NotLab', 56: 'NotLab', 57: 'NotLab', 58: 'NotLab'}
group_by_cat {'NotLab': [0, 1, 2, 3, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30], 'CSE Lab': [4, 5], 'ECE Lab': [18

In [5]:

from collections import defaultdict

course_student = defaultdict(list)
intersection_pairs = set()
intersection_triplets = set()
for i in range(n_students):
    courses_taken = get_courses_student(i)
    for x in courses_taken:
        course_student[x].append(i)
    for j in range(len(courses_taken)):
        for k in range(j+1, len(courses_taken)):
            intersection_pairs.add((courses_taken[j], courses_taken[k]))
            for l in range(k+1, len(courses_taken)):
                intersection_triplets.add((courses_taken[j], courses_taken[k], courses_taken[l]))
    
intersection_pairs = list(intersection_pairs)
n_intersection_pairs = len(intersection_pairs)
# print(intersection_pairs)
# print(course_student)
# print(intersection_triplets)



In [6]:
def check_for_intersection(c1:str, c2:str) -> bool:
    return (get_course_id(c1), get_course_id(c2)) in intersection_pairs

# print(check_for_intersection('AIM 101', 'CSE102'))
l1 = ['AIM 101', 'AIM 821', 'AIM 832', 'COM 605', 'ECE 212P', 'DHS 108']
l2 =['DHS 314','CSE 754','AIM 843','DAS 101P-A','DAS 101P-B','EGC 123-A','EGC 123-B','ECE 212']

l3=['DAS 101-A','DAS 101-B','AIM 831','ECE 303']
l4=['CSE 102 P-A','DAS 605','ECE 303P','VLS 804','DHS 306']

l5=['AIM 704', 'CSE 102-A', 'CSE 102-B', 'CSE 212', 'CSE 718', 'ECE 304']
l6=['CSE 102 P-B', 'DAS 839', 'DHS 305']

l7 = ['CSE 104-A','CSE 104-B','CSE 704','DHS 110']
l8 = ['AIM 845', 'AIM 846', 'EGC 301', 'DHS 308']

l9 = ['AMS 401','AMS 402','AMS 403','AMS 404', 'CSE 606', 'CSE 836','DHS 101B-A','DHS 101B-B']
l10 = ['CSE 816', 'AIM 102', 'VLS 603', 'DHS 315']

count=0
l=l9 +l10
for i in l:
    for j in l:
        if i != j:
            if check_for_intersection(i,j):
                count+=1
print(count//2)

print(len(l1+l2+l3+l4+l5+l6+l7+l8+l9+l10))

2
52


In [7]:
def get_intersection_of_courses(c1, c2):
    return (set(course_student[c1]) & set(course_student[c2]))
# print(course_student[4])
# print(course_student[24])
# print(get_intersection_of_courses(24, 4))

In [8]:
course_batch = {}

for c in range(n_courses):
    batch = {}
    for s in course_student[c]:
        # enrollment is a dataframe help me extract the same
        if(enrollment[enrollment['EmailID'] == students[s]]['Batch'].values[0] in batch):
            batch[enrollment[enrollment['EmailID'] == students[s]]['Batch'].values[0]] += 1
        else :
            batch[enrollment[enrollment['EmailID'] == students[s]]['Batch'].values[0]] = 1
    mx = 0
    for key in batch:
        if(batch[key] > mx):
            mx = batch[key]
            mxbatch = key
    course_batch[c] = mxbatch
def get_batch(course_id: int) -> int:
    return course_batch[course_id]

print(get_batch(3))

2022-27iMtech


## Faculty

In [9]:
faculty_cols = {'Name': str, 'Course': str, 'Start': str, 'End': str,
                'PreferredSlot': str, 'Flexibility': int}
faculty = pd.read_csv('faculty_master.csv', dtype=faculty_cols)